In [7]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer   # 결측치값 대체.
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline

In [8]:
df = pd.read_csv("../data/final_dataset.csv")
X = df.drop(columns='target').values
y = df['target']

X.shape,  y.shape

((32545, 16), (32545,))

In [9]:
fe_transformer = ColumnTransformer([
    ("category_ohe", OneHotEncoder(), [0, 1, 3, 4, 5, 6, 7, 8, 10,15]),# feature의 index로 지정.   # index는 앞에 했던 배열로 적용해줘야함.
    ("number_scaler", StandardScaler(), [9,11,12,13,14])    #feature Scaling은 연속형끼리 같은 방식을 사용 (standard or MinMax 중 택1)
])
### DataFrame이 입력일 경우 컬럼명이나 컬럼 index를 지정할 수 있다.
### ndarray가 입력일 경우 컬럼(feature) index를 지정.
new_merged_df = fe_transformer.fit_transform(X)
print(new_merged_df)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 488169 stored elements and shape (32545, 60)>
  Coords	Values
  (0, 0)	1.0
  (0, 8)	1.0
  (0, 12)	1.0
  (0, 13)	1.0
  (0, 29)	1.0
  (0, 40)	1.0
  (0, 43)	1.0
  (0, 44)	1.0
  (0, 51)	1.0
  (0, 53)	1.0
  (0, 55)	3.905125197007625
  (0, 56)	-1.8185928989610989
  (0, 57)	0.6671221427396934
  (0, 58)	-0.1670317117402033
  (0, 59)	0.7100890620793591
  (1, 0)	1.0
  (1, 8)	1.0
  (1, 11)	1.0
  (1, 19)	1.0
  (1, 29)	1.0
  (1, 33)	1.0
  (1, 42)	1.0
  (1, 44)	1.0
  (1, 51)	1.0
  (1, 53)	1.0
  :	:
  (32543, 40)	1.0
  (32543, 42)	1.0
  (32543, 44)	1.0
  (32543, 51)	1.0
  (32543, 54)	1.0
  (32543, 55)	-1.211159957215567
  (32543, 56)	0.4144516877944941
  (32543, 57)	-1.487791864498921
  (32543, 58)	-0.5562260670917427
  (32543, 59)	1.0137563256706723
  (32544, 6)	1.0
  (32544, 10)	1.0
  (32544, 11)	1.0
  (32544, 25)	1.0
  (32544, 29)	1.0
  (32544, 36)	1.0
  (32544, 42)	1.0
  (32544, 44)	1.0
  (32544, 51)	1.0
  (32544, 53)	1.0
  (32544, 55)

In [10]:
X = new_merged_df
X

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 488169 stored elements and shape (32545, 60)>

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=0)

In [12]:
from catboost import CatBoostClassifier, Pool
from scipy.stats import randint, loguniform
import numpy as np
import pandas as pd

# 1. 데이터셋 준비 (X_train, y_train / X_test, y_test 등)
# 이미 X_train, y_train 준비된 상태라고 가정

# 2. 파라미터 샘플링 범위 설정
param_grid = {
    'iterations': randint(100, 300),
    'depth': randint(1, 5),
    'learning_rate': loguniform(1e-3, 0.1),
    'min_data_in_leaf': randint(10, 40),
    'grow_policy': ['SymmetricTree', 'Lossguide', 'Depthwise']
}

# 3. CatBoostClassifier 인스턴스 생성
model_cat = CatBoostClassifier(
    verbose=0,
    random_seed=42,
    task_type='CPU',
    loss_function='Logloss',
    eval_metric='Accuracy'
)

# 4. 데이터 Pool 객체로 래핑 (CatBoost 내부 최적화에 사용)
train_pool = Pool(data=X_train, label=y_train)

# 5. random search 수행 (n_iter = 시도 횟수)
model_cat.randomized_search(param_grid,
                            X=train_pool,
                            cv=3,
                            n_iter=10,
                            partition_random_seed=42,
                            calc_cv_statistics=True,
                            refit=True,
                            shuffle=True,
                            stratified=True,
                            verbose=1)

# 6. 최적 파라미터로 전체 데이터 학습
model_cat.fit(X_train, y_train, verbose=0)

# 7. 예측 및 평가
y_pred = model_cat.predict(X_test)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))



bestTest = 0.8691110201
bestIteration = 148

0:	loss: 0.8691110	best: 0.8691110 (0)	total: 1.11s	remaining: 9.96s

bestTest = 0.8633756657
bestIteration = 95

1:	loss: 0.8633757	best: 0.8691110 (0)	total: 1.78s	remaining: 7.13s

bestTest = 0.8816058992
bestIteration = 105

2:	loss: 0.8816059	best: 0.8816059 (2)	total: 2.32s	remaining: 5.42s

bestTest = 0.8869315854
bestIteration = 172

3:	loss: 0.8869316	best: 0.8869316 (3)	total: 3.44s	remaining: 5.16s

bestTest = 0.8857025809
bestIteration = 273

4:	loss: 0.8857026	best: 0.8869316 (3)	total: 4.91s	remaining: 4.91s

bestTest = 0.8969684556
bestIteration = 161

5:	loss: 0.8969685	best: 0.8969685 (5)	total: 5.88s	remaining: 3.92s

bestTest = 0.878533388
bestIteration = 281

6:	loss: 0.8785334	best: 0.8969685 (5)	total: 8.12s	remaining: 3.48s

bestTest = 0.8816058992
bestIteration = 132

7:	loss: 0.8816059	best: 0.8969685 (5)	total: 9.05s	remaining: 2.26s

bestTest = 0.890823433
bestIteration = 246

8:	loss: 0.8908234	best: 0.8969685 (5